# Load The Data

In [ ]:
%cd '/content/drive/My Drive/Colab_Notebooks/'

In [ ]:
%ls

In [ ]:
!unzip 'cell_images.zip'

In [ ]:
import cv2
import matplotlib.pyplot as plt 
import seaborn as sns
import os
import tensorflow as tf
import numpy as np
import datetime
import itertools
from tensorflow.keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Activation, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam,SGD
from keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.callbacks import TensorBoard
from keras.models import load_model
from sklearn.metrics import confusion_matrix,classification_report,f1_score, precision_score, recall_score

In [ ]:
parasitized = os.listdir('/content/drive/My Drive/Colab_Notebooks/cell_images/Parasitized/')
print(parasitized[:5])

uninfected = os.listdir('/content/drive/My Drive/Colab_Notebooks/cell_images/Uninfected/')
print(uninfected[:5])

In [ ]:
#visualitation
plt.figure(figsize = (20,20))
for i in range(10):
    plt.subplot(2, 5, i+1)
    img = cv2.imread('/content/drive/My Drive/Colab_Notebooks/cell_images/Parasitized' + "/" + parasitized[i])
    plt.imshow(img)
    plt.title('PARASITIZED : 1')
    plt.tight_layout()
plt.show()

In [ ]:
#visualitation
plt.figure(figsize = (20,20))
for i in range(10):
    plt.subplot(2, 5, i+1)
    img = cv2.imread('/content/drive/My Drive/Colab_Notebooks/cell_images/Uninfected' + "/" + uninfected[i])
    plt.imshow(img)
    plt.title('UNINFECTED : 0')
    plt.tight_layout()
plt.show()

## Load images to array and labelling

In [ ]:
data=[]
labels=[]

for img in parasitized:
  try:
    image=plt.imread('/content/drive/My Drive/Colab_Notebooks/cell_images/Parasitized/'+img)
    image_resize = cv2.resize(image,(44,44))
    img_array = img_to_array(image_resize)
    data.append(img_array)
    labels.append(1)
  except:
    None

for img in uninfected:
  try:
    image=plt.imread('/content/drive/My Drive/Colab_Notebooks/cell_images/Uninfected/'+img)
    image_resize = cv2.resize(image,(44,44))
    img_array = img_to_array(image_resize)
    data.append(img_array)
    labels.append(0)
  except:
    None

plt.imshow(data[100])
plt.show()                    

In [ ]:
print(len(data))
print(len(labels))

## Split Data 90:10 ratio

In [ ]:
image_data = np.array(data)
labels = np.array(labels)

In [ ]:
idx = np.arange(image_data.shape[0])
np.random.shuffle(idx)
image_data = image_data[idx]
labels = labels[idx]

In [ ]:
from sklearn.model_selection import train_test_split
feature_train, feature_test, label_train, label_test = train_test_split(image_data, labels, test_size = 0.10, random_state = 10)

label_train = to_categorical(label_train)
label_test = to_categorical(label_test)

print(f'SHAPE OF TRAINING IMAGE DATA : {feature_train.shape}')
print(f'SHAPE OF TESTING IMAGE DATA : {feature_test.shape}')
print(f'SHAPE OF TRAINING LABELS : {label_train.shape}')
print(f'SHAPE OF TESTING LABELS : {label_test.shape}')

# Build The CNN Net

In [ ]:
model= Sequential()

model.add(Conv2D(filters=32,kernel_size=(5,5),strides=1,input_shape = (44,44,3)))
model.add(Activation('relu'))
model.add(Conv2D(filters=32,kernel_size=(5,5),strides=1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(5,5),strides=2))
model2.add(BatchNormalization(axis=-1))
model2.add(Dropout(0.5))

model.add(Conv2D(filters=64,kernel_size=(5,5),strides=1))
model.add(Activation('relu'))
model.add(Conv2D(filters=64,kernel_size=(5,5),strides=1))
model.add(MaxPooling2D(pool_size=(3,3),strides=2))
model2.add(BatchNormalization(axis=-1))
model2.add(Dropout(0.5))

model.add(Conv2D(filters=128,kernel_size=(5,5),strides=1,padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(filters=256,kernel_size=(4,4),strides=1,padding='same'))
model2.add(BatchNormalization(axis=-1))
model2.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(units=256,activation='relu'))
model.add(Dense(units=256,activation='relu'))
model.add(Dense(units=2, activation='softmax'))

model.summary()

## Visualize the CNN filters and Feature Map

In [ ]:
# summarize filter shapes
for layer in model_.layers:
	# check for convolutional layer
	if 'conv' not in layer.name:
		continue
  # get filter weights
	filters, biases = layer.get_weights()
	print(layer.name, filters.shape)

In [ ]:
filters, biases = model_.layers[2].get_weights()
# normalize filter values to 0-1 so we can visualize them
f_min, f_max = filters.min(), filters.max()
filters = (filters - f_min) / (f_max - f_min)
# plot first few filters
n_filters, ix = 6, 1
for i in range(n_filters):
	# get the filter
	f = filters[:, :, :, i]
	# plot each channel separately
	for j in range(3):
		# specify subplot and turn of axis
		ax = plt.subplot(n_filters, 3, ix)
		ax.set_xticks([])
		ax.set_yticks([])
		# plot filter channel in grayscale
		plt.imshow(f[:, :, j], cmap='gray')
		ix += 1
# show the figure
plt.show()

In [ ]:
from keras.models import Model
layer_outputs = [layer.output for layer in model_.layers]
activation_model = Model(inputs=model_.input, outputs=layer_outputs)
activations = activation_model.predict(feature_train[1557].reshape(1,44,44,3))
 
def display_activation(activations, col_size, row_size, act_index): 
    activation = activations[act_index]
    activation_index=0
    fig, ax = plt.subplots(row_size, col_size, figsize=(row_size*2.5,col_size*1.5))
    for row in range(0,row_size):
        for col in range(0,col_size):
            ax[row][col].imshow(activation[0, :, :, activation_index], cmap='gray')
            activation_index += 1

In [ ]:
plt.imshow(feature_train[1557][:,:,0]);

In [ ]:
display_activation(activations, 8, 4, 1)

In [ ]:
display_activation(activations, 8, 8, 2)

## Setting up Callbacks

In [ ]:
filepath='weights.h5'
my_callbacks = [
                tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',patience =10),
                tf.keras.callbacks.ModelCheckpoint(filepath , monitor='val_accuracy',verbose=1,save_best_only=True),
]

logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
my_callbacks.append(TensorBoard(logdir, histogram_freq=1))

print(my_callbacks)

## Start Training

In [ ]:
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(feature_train,label_train, validation_data=(feature_test,label_test), callbacks = my_callbacks, epochs=100,verbose=1)

#Load Tensorboard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir '/content/drive/My Drive/Colab_Notebooks/baru/logs1'

# Evaluate the model

In [ ]:
model_ = load_model('/content/drive/My Drive/Colab_Notebooks/weights.h5')

In [ ]:
model_.summary()

In [ ]:
loss, accuracy = model_.evaluate(feature_test,label_test, verbose=1)

print("Test accuracy:", accuracy)
print("Test loss:", loss)

In [ ]:
predictions = model_.predict(feature_test)
print(np.round(predictions))
print(classification_report(np.argmax(label_test, axis = 1), np.argmax(predictions, axis = 1)))
cm = confusion_matrix(np.argmax(label_test, axis = 1), np.argmax(predictions, axis = 1))

print('F1_score   on test set: {:.4f}'.format(f1_score(np.argmax(label_test, axis = 1), np.argmax(predictions, axis = 1), average="macro")))

print('Precision  on test set: {:.4f}'.format(precision_score(np.argmax(label_test, axis = 1), np.argmax(predictions, axis = 1), average="macro")))

print('Recall     on test set: {:.4f}'.format(recall_score(np.argmax(label_test, axis = 1), np.argmax(predictions, axis = 1), average="macro")))

In [ ]:
def plot_confusion_matrix(cm,classes,normalize=False, title='Confusion Matrix', cmap=plt.cm.Blues):
  plt.imshow(cm,interpolation='nearest',cmap=cmap)
  plt.title(title)
  plt.colorbar()
  tick_marks = np.arange(len(classes))
  plt.xticks(tick_marks, classes, rotation=45)
  plt.yticks(tick_marks, classes)

  if normalize:
    cm = cm.astype('float')/cm.sum(axis=1)[:, np.newaxis]
    print("normalized confusion matrix")
  else:
    print("Confusion Matrix")
  
  print(cm)
  thresh = cm.max()/2.
  for i,j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, cm[i,j], horizontalalignment="center",color="white" if cm[i,j]>thresh else "black")
  

  plt.tight_layout()
  plt.ylabel('True Label')
  plt.xlabel('Predicted Label')


cm_plot_labels = ['Uninfected','Parasitized']
plot_confusion_matrix(cm=cm, classes=cm_plot_labels,title='Confusion Matrix')

## Test to one Image

In [ ]:
testing_path="/content/drive/My Drive/Colab_Notebooks/cell_images/Parasitized/C100P61ThinF_IMG_20150918_144348_cell_143.png"
img= image.load_img(testing_path,target_size=(44,44))
plt.imshow(img)

x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)
images=np.vstack([x])
val=model_.predict(images)
result = np.argmax(val[0]).tolist()
print('Class : '+str(result))